In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers datasets torch evaluate --quiet

In [ ]:
# Gerekli kütüphaneleri import et
import torch
import json
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq

In [ ]:
# Eğitim ve doğrulama veri dosyalarının yollarını tanımlayın
train_data_path = "/content/drive/MyDrive/llm_qa/t5/t5_formatted_data_train.json"
dev_data_path = "/content/drive/MyDrive/llm_qa/t5/t5_formatted_data.json"

# Eğitim ve doğrulama verilerini yükleyin
with open(train_data_path, "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open(dev_data_path, "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# Verilerin doğru yüklendiğini kontrol edelim
print("Eğitim verisi ilk 2 örnek:")
print(train_data[:2])
print("\nDoğrulama verisi ilk 2 örnek:")
print(dev_data[:2])

In [ ]:
from datasets import Dataset, DatasetDict

# Hugging Face Dataset formatına dönüştürme
train_dataset = Dataset.from_list(train_data)
dev_dataset = Dataset.from_list(dev_data)

# Datasetleri birleştirerek bir DatasetDict oluşturma
dataset = DatasetDict({
    "train": train_dataset,
    "validation": dev_dataset
})

# Datasetlerin örneklerini kontrol edelim
print("Eğitim veri seti örneği:")
print(dataset["train"][0])
print("\nDoğrulama veri seti örneği:")
print(dataset["validation"][0])


Eğitim veri seti örneği:
{'input': "context: Rollo'nun gelişinden önce popülasyonları Picardy'den veya 'Franklar' olarak nitelendirilen Île-de-France'den farklı değildi. Daha önceki Viking yerleşimcileri 880'lerde gelmeye başlamıştı ancak doğudaki koloniler (Roumois ve Pays de Caux) arasında alçak Seine vadisi çevresinde ve batıdaki Cotentin Yarımadası'ndaki koloniler arasında bölündü ve hemen hemen hiçbir yabancı yerleşimci olmadan nüfusun aynı kaldığı geleneksel pagii ile ayrıldılar.  Normandiya ve Atlantik kıyılarının bir kısmına baskın düzenleyen ve yerleşen Rollo'nun muhalifleri Norveçliler, İskandinav-İskoçyalılar, Orkney Vikingler, muhtemelen İsveçliler ve Norveç kontrolündeki İngiliz Danelaw'lardan Anglo Danimarkalıları içermektedir.  question: Kim geldiğinde orijinal viking yerleşimcilerine ortak bir kimlik vermiştir?", 'output': 'Rollo'}

Doğrulama veri seti örneği:
{'input': "context: Osman Bey 1258 yılında Söğüt’te doğdu. Osman Bey 1 Ağustos 1326’da Bursa’da hayatını kaybet

In [ ]:
from transformers import T5Tokenizer

# Tokenizer yükleme
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenizasyon fonksiyonu
def preprocess_data(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Eğitim veri setini tokenize etme
tokenized_train_dataset = dataset["train"].map(
    preprocess_data,
    batched=True,
    remove_columns=["input", "output"]
)

# Doğrulama veri setini tokenize etme
tokenized_dev_dataset = dataset["validation"].map(
    preprocess_data,
    batched=True,
    remove_columns=["input", "output"]
)

# Tokenize edilmiş örnekleri kontrol edelim
print("Tokenize edilmiş veri seti örneği (Eğitim):")
print(tokenized_train_dataset[0])

print("Tokenize edilmiş veri seti örneği (Doğrulama):")
print(tokenized_dev_dataset[0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/14221 [00:00<?, ? examples/s]

Map:   0%|          | 0/1330 [00:00<?, ? examples/s]

Tokenize edilmiş veri seti örneği (Eğitim):
{'input_ids': [2625, 10, 6070, 32, 31, 29, 202, 7568, 23, 1816, 29, 537, 3, 11431, 565, 2783, 13198, 9, 7, 63, 106, 40, 291, 2, 11000, 986, 63, 31, 537, 3, 162, 63, 9, 3, 31, 371, 6254, 40, 291, 31, 3, 17401, 1639, 3, 7980, 40, 35, 12594, 173, 35, 3, 10997, 109, 18, 221, 18, 371, 5219, 31, 537, 623, 157, 40, 2, 20, 2, 173, 26, 23, 5, 878, 1024, 3, 11431, 565, 2168, 19476, 3, 7975, 109, 1816, 51, 75, 173, 49, 23, 505, 2079, 31, 1171, 221, 7568, 51, 13370, 4698, 1314, 40, 265, 2, 1314, 17, 2, 46, 658, 157, 103, 2, 76, 26, 11259, 3, 11292, 106, 173, 49, 41, 448, 1063, 16661, 7, 3, 162, 5077, 7, 20, 205, 1724, 61, 3, 2551, 7, 2, 727, 9, 491, 11666, 157, 679, 630, 409, 26, 159, 23, 3, 8970, 15, 10859, 7, 77, 221, 3, 162, 3795, 2, 26, 11259, 638, 4669, 77, 4701, 52, 2, 11374, 9, 7, 2, 31, 727, 11259, 3, 11292, 106, 173, 49, 3, 2551, 7, 2, 727, 9, 3, 115, 7898, 1272, 727, 1272, 3, 162, 3, 88, 904, 3, 88, 904, 7102, 8970, 8781, 3, 63, 9, 3478, 75, 2,

In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

# Modeli yükleme
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Eğitim parametreleri
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/llm_qa/t5/custom-t5-turkish",          # Model çıktı dosyalarının kaydedileceği klasör
    evaluation_strategy="steps",                    # Değerlendirme sıklığı (adım bazlı)
    eval_steps=500,                                 # Kaç adımda bir değerlendirme yapılacağı
    learning_rate=5e-5,                             # Öğrenme oranı
    per_device_train_batch_size=16,                 # Her cihaz için eğitim batch boyutu
    per_device_eval_batch_size=16,                  # Her cihaz için değerlendirme batch boyutu
    num_train_epochs=9,                             # Eğitim boyunca epoch sayısı
    weight_decay=0.01,                              # Ağırlık azaltma (regularization)
    save_steps=500,                                 # Model kaydedilecek adım sayısı
    save_total_limit=2,                             # Maksimum kaç model kaydedileceği
    logging_dir="./logs",                           # Log dosyalarının kaydedileceği klasör
    logging_steps=100,                              # Kaç adımda bir log kaydedileceği
    load_best_model_at_end=True,                    # En iyi modeli eğitim sonunda yükler
    push_to_hub=False                               # Hugging Face Hub'a yükleme seçeneği (opsiyonel)
)

print("Model ve eğitim parametreleri tanımlandı.")

# Trainer nesnesini tanımlayın
trainer = Trainer(
    model=model,                          # Kullanılacak model
    args=training_args,                   # Eğitim parametreleri
    train_dataset=tokenized_train_dataset,  # Eğitim veri seti
    eval_dataset=tokenized_dev_dataset     # Değerlendirme veri seti
)

# Model eğitimi
trainer.train()

print("Model eğitimi tamamlandı!")

Model ve eğitim parametreleri tanımlandı.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer nesnesini tanımlayın
trainer = Trainer(
    model=model,                          # Kullanılacak model
    args=training_args,                   # Eğitim parametreleri
    train_dataset=tokenized_train_dataset,  # Eğitim veri seti
    eval_dataset=tokenized_dev_dataset     # Değerlendirme veri seti
)

# Model eğitimi
trainer.train()

print("Model eğitimi tamamlandı!")


Step,Training Loss,Validation Loss
500,0.189600,0.162510
1000,0.156100,0.154389
1500,0.162500,0.151883
2000,0.139500,0.147098
2500,0.145200,0.145407
3000,0.141100,0.142746
3500,0.139500,0.142876
4000,0.124800,0.141755
4500,0.129000,0.140528
5000,0.126200,0.140159


Step,Training Loss,Validation Loss
500,0.189600,0.162510
1000,0.156100,0.154389
1500,0.162500,0.151883
2000,0.139500,0.147098
2500,0.145200,0.145407
3000,0.141100,0.142746
3500,0.139500,0.142876
4000,0.124800,0.141755
4500,0.129000,0.140528
5000,0.126200,0.140159


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Model eğitimi tamamlandı!


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint_path2 = r"/content/drive/MyDrive/llm_qa/t5/t5-turkish-qa/checkpoint-2667" # En son checkpoint klasörünüzün adını buraya yazın
checkpoint_path = r"/content/drive/MyDrive/llm_qa/llm_final_project/abdulkerim_akan/custom-t5-turkish/checkpoint-8001" # En son checkpoint klasörünüzün adını buraya yazın

# Tokenizer ve modeli yükle
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)

# Test verisi
context = """Osman Bey 1258 yılında Sögüt’te doğdu. Osman Bey 1 Ağustos 1326’da Bursa’da hayatını kaybetmiştir.1281 yılında Osman Bey 23 yaşında iken Ahi teşkilatından olan Şeyh Edebali’nin kızı Malhun Hatun ile evlendi.Bu evlilikten daha sonra Osmanlı Devleti’nin başına geçecek olan Orhan Gazi doğdu.1281 yılında Osman Beyin babası Ertuğrul Bey 90 yaşında  vefat etmiştir.1326’da Osman Bey, Bursa’yı kuşattı. Fakat Osman beyin rahatsızlanması üzerine kuşatmaya Orhan Bey devam etti. Bursa alındıktan sonra başkent yapılmıştır.Osman Gazi son yıllarında yaşının ilerlemesi ve gut hastalığı yüzünden beylik idaresini oğlu olan Orhan Bey'e bırakmıştı.Osmanlı Beyliğinin ilk fethettiği ada İmralı Adasıdır. İmralı Adası 1308 yılında Osman Bey tarafından alınmıştır.İlk Osmanlı parası Osman Bey tarafından bakır olarak akçe adı ile 1324 yılında bastırılmıştır.Osmanlı Beyliğinin ilk başkenti Söğüttür.Osmanlı tarihinde ilk savaş, 1284 yılında Bizans tekfurlarıyla yapılan Ermeni Beli savaşıdır.Osman Beyin ele geçirdiği ilk kale 1285 yılında fethedilen Kolca Hisar Kalesi’dir.Osmanlı beyliğinin ilk kadısı Osman Bey döneminde atanan Dursun Fakih’tir.Osman Bey 1288 yılında Karacahisarı fethetti. Osman Bey 1299 yılında Bilecik'i fethetti.Osman Gazi, babası Ertuğrul Gazi'den yaklaşık 4.800 kilometrekare olarak devraldığı Osmanlı toprağını oğlu Orhan Gazi'ye 16.000 kilometrekare olarak devretmiştir.Osman Bey'in vefatı sonrası yerine Orhan Bey geçti"""

question = "Osman Bey nerde dogdu?"

# Soruyu ve bağlamı birleştir
input_text = f"question: {question} context: {context}"

# Veriyi tokenleştir
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    max_length=512,
    truncation=True,
    padding="max_length"
)

# Modeli tahmin için çalıştır
outputs = model.generate(inputs["input_ids"], max_length=64, num_beams=4)

# Tahmin edilen cevabı çözümleme
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Soru: {question}")
print(f"Cevap: {answer}")


Soru: Osman Bey nerde dogdu?
Cevap: Osmanl paras Osman Bey tarafndan bakr olarak ak ak evlilikten daha sonra Osmanl Devleti’nin babas Erturul Bey


In [ ]:
!ls "/content/drive/MyDrive/llm_qa/t5/t5-turkish-qa/checkpoint-2667"
!ls checkpoint_path

added_tokens.json	model.safetensors  scheduler.pt		    tokenizer_config.json
config.json		optimizer.pt	   special_tokens_map.json  trainer_state.json
generation_config.json	rng_state.pth	   spiece.model		    training_args.bin
ls: cannot access 'checkpoint_path': No such file or directory


In [ ]:
# Model ve Tokenizer'ı yükleyin
model_name = "dbmdz/bert-base-turkish-cased"
save_path = "/"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

NameError: name 'tokenizer' is not defined

In [ ]:
!rsync -av --ignore-existing "/content/drive/MyDrive/llm_qa/t5/t5-turkish-qa/checkpoint-2667/" "/content/drive/MyDrive/llm_qa/t5/custom-t5-turkish/checkpoint-8001/"

sending incremental file list
./
added_tokens.json
special_tokens_map.json
spiece.model
tokenizer_config.json

sent 818,318 bytes  received 95 bytes  545,608.67 bytes/sec
total size is 727,051,688  speedup is 888.37
